<a href="https://colab.research.google.com/github/kimyeulin/kimyeulin.github.io/blob/master/nextSentencePred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <세팅>

#### Google Cloud Platform TPU 연결

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.30.214.178:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3255606024744368659),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15199437583002770786),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8991041629631749352),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11272170744900025088),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8721509241216896569),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5799266972572538166),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 4741552956328893229),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15630226455823861999),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 100586673636

#### 기타 라이브러리 import 및 BERT 설치

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0601 15:59:56.300253 139694058526592 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

#### 저장 디렉토리 지정

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'nextSentence'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'bert_open_1' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: gs://bert_open_1/nextSentence *****


# <데이터 불러오기 및 전처리>

In [0]:
from tensorflow import keras
import os
import re
from sklearn.model_selection import train_test_split

Task #2: Next Sentence Prediction (NSP)
Many important downstream tasks such as Question Answering (QA) and Natural Language Inference (NLI) are based on understanding the relationship between two sentences, which is not directly captured by language modeling. In order
to train a model that understands sentence relationships, we pre-train for a binarized next sentence prediction task that can be trivially generated from any monolingual corpus. Specifically,
when choosing the sentences A and B for each pretraining example, ***50% of the time B is the actual
next sentence that follows A (labeled as IsNext),
and 50% of the time it is a random sentence from
the corpus (labeled as NotNext).*** 

In [0]:
# Load all files from a directory in a DataFrame.
'''
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)
'''
################################################################################
################################################################################
def load_directory_data(directory):
    dataset = pd.read_csv(directory)
    types      = dataset["유형"]
    paragraphs = dataset["지문"]
    answers    = dataset["답"]
    
    dataset_ = []
    for type, paragraph, answer in zip(types,paragraphs,answers):
        paragraph = [sentence.strip() for sentence in paragraph.split('.')]
        # paragraph = [sentence for sentence in sentences if not sentence =='']
        data_ = {}
        data_["type"]      = type
        data_["paragraph"] = paragraph
        data_["answer"]    = answer
        dataset_.append(data_)
        
    type         = []
    prevSentence = []
    nextSentence = []
    sentenceNum  = []
    sentenceMax  = []
    answer       = []
    for data_ in dataset_:
        prev = 0 
        next = 1
        sentenceMax_=len(data_["paragraph"])
        while next < sentenceMax_:
            type.append(data_["type"])
            prevSentence.append(data_["paragraph"][prev])
            nextSentence.append(data_["paragraph"][next])
            sentenceNum.append(next)
            sentenceMax.append(sentenceMax_-1)
            answer.append(data_["answer"])#정답
            prev=prev+1 
            next=next+1

    data = {}
    data["type"]         = type
    data["prevSentence"] = prevSentence
    data["nextSentence"] = nextSentence
    data["sentenceNum"]  = sentenceNum
    data["sentenceMax"]  = sentenceMax
    data["answer"]       = answer
    return pd.DataFrame.from_dict(data)
################################################################################
################################################################################
# Merge positive and negative examples, add a polarity column and shuffle.
'''
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)
'''
################################################################################
################################################################################
def load_dataset(directory): # 수정 필요
  dataset = load_directory_data(directory)
  dataset_t, dataset_f = train_test_split(dataset, test_size=0.5)
  nextSentence_rand = list(
                          dataset[ dataset["nextSentence"] != '' ]
                          ["nextSentence"]
                          .sample( len(dataset_f) )
                          )
  del dataset_f["nextSentence"]
  dataset_f["nextSentence"] = nextSentence_rand
  dataset_t["tf"] = 1
  dataset_f["tf"] = 0 
  return pd.concat([dataset_t, dataset_f]).sample(frac=1).reset_index(drop=True)
  

  
################################################################################
################################################################################
# Download and process the dataset files.
'''
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df
'''

'\ndef download_and_load_datasets(force_download=False):\n  dataset = tf.keras.utils.get_file(\n      fname="aclImdb.tar.gz", \n      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", \n      extract=True)\n  \n  train_df = load_dataset(os.path.join(os.path.dirname(dataset), \n                                       "aclImdb", "train"))\n  test_df = load_dataset(os.path.join(os.path.dirname(dataset), \n                                      "aclImdb", "test"))\n  \n  return train_df, test_df\n'

In [0]:
# 파일을 콜랩에 업로드
from google.colab import files
uploaded = files.upload()

Saving dataset_except_flow_.csv to dataset_except_flow_ (2).csv


In [0]:
train = load_dataset('dataset_except_flow_.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [0]:
train = load_directory_data('dataset_except_flow_.csv')

In [0]:
train

,type,prevSentence,nextSentence,sentenceNum,sentenceMax,answer
0,목적,Dear Mr,Reese,1,9,3
1,목적,Reese,"A few days ago, I submitted my application and...",2,9,3
2,목적,"A few days ago, I submitted my application and...","However, I would like to change my recipe if i...",3,9,3
3,목적,"However, I would like to change my recipe if i...","I have checked the website again, but I could ...",4,9,3
4,목적,"I have checked the website again, but I could ...",I couldn’t see any information about changing ...,5,9,3
5,목적,I couldn’t see any information about changing ...,"I have just created a great new recipe, and I ...",6,9,3
6,목적,"I have just created a great new recipe, and I ...",Please let me know if I can change my submitte...,7,9,3
7,목적,Please let me know if I can change my submitte...,I look forward to your response,8,9,3
8,목적,I look forward to your response,"Best Regards, Sophia Walker",9,9,3
9,심경변화,The waves were perfect for surfing,"Dave, however, just could not stay on his board",1,11,1


In [0]:
DATA_COLUMN_A = 'prevSentence'
DATA_COLUMN_B = 'nextSentence'
LABEL_COLUMN  = 'tf'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0,1]

#### input 형태 지정

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                             text_a = x[DATA_COLUMN_A], 
                                                                             text_b = x[DATA_COLUMN_B], 
                                                                             label  = x[LABEL_COLUMN]),
                                                                             axis = 1)

'''
test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                           text_a = x[DATA_COLUMN_A], 
                                                                           text_b = x[DATA_COLUMN_B], 
                                                                           label  = x[LABEL_COLUMN]),
                                                                           axis = 1)
'''

'\ntest_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, \n                                                                           text_a = x[DATA_COLUMN_A], \n                                                                           text_b = x[DATA_COLUMN_B], \n                                                                           label  = x[LABEL_COLUMN]),\n                                                                           axis = 1)\n'

#### BERT tokenizer import

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0601 16:00:46.128724 139694058526592 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0601 16:00:48.627878 139694058526592 saver.py:1483] Saver not created because there are no variables in the graph to restore


#### tokenizing → feature embedding

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
#test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 2451


I0601 16:00:49.279774 139694058526592 run_classifier.py:774] Writing example 0 of 2451


INFO:tensorflow:*** Example ***


I0601 16:00:49.285800 139694058526592 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0601 16:00:49.391983 139694058526592 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0601 16:00:49.394200 139694058526592 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0601 16:00:49.396444 139694058526592 run_classifier.py:468] label: 1 (id = 1)


# <모델 정의 (Fine Tuning)>

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, 
                 num_labels):
  """Creates a classification model."""
  # 기본세팅
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence. → [CLS] 토큰에 달리는 output?
  # Use "sequence_outputs" for token-level output. → 몇 번째 time에 달지는 어떻게 설정?
  # bert_outputs가 BERT의 기본 학습 출력값인 듯.
  # bert_outputs["출력 종류"] → 출력 종류는 "pooled_output" / "sequence_outputs" 두 가지 뿐? 
  output_layer = bert_outputs["pooled_output"]
  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels],
      initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting: # 입력으로 is_predicting 값을 1로 주면 predict까지 수행.
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function using the passed parameters for num_labels, learning_rate, etc.

def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'gs://bert_open_1/nextSentence', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0cc70c22b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0601 16:00:58.573490 139694058526592 estimator.py:201] Using config: {'_model_dir': 'gs://bert_open_1/nextSentence', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0cc70c22b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0601 16:01:03.665176 139694058526592 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0601 16:01:07.403538 139694058526592 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 229 into gs://bert_open_1/nextSentence/model.ckpt.


I0601 18:57:50.609084 139694058526592 basic_session_run_hooks.py:594] Saving checkpoints for 229 into gs://bert_open_1/nextSentence/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0047090002.


I0601 18:58:22.823701 139694058526592 estimator.py:359] Loss for final step: 0.0047090002.


Training took time  2:57:20.869324


In [0]:
def getPrediction(sentences):
  labels = ["NotNext", "IsNext"]
  # in_sentences = [ [text_a,text_b], [text_a,text_b], [text_a,text_b], [text_a,text_b], ... ]
  input_examples = [run_classifier.InputExample(guid="", text_a = sentence[0], text_b = sentence[1], label = 0) for sentence in sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(sentences, predictions)]

In [0]:
getPrediction(['When photography came along in the nineteenth century, painting was put in crisis', 'The photograph, it seemed, did the work of imitating nature better than the painter ever could'])

INFO:tensorflow:Writing example 0 of 2


I0601 19:52:49.674448 139694058526592 run_classifier.py:774] Writing example 0 of 2


INFO:tensorflow:*** Example ***


I0601 19:52:49.678886 139694058526592 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0601 19:53:17.611292 139694058526592 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0601 19:53:17.788678 139694058526592 session_manager.py:493] Done running local_init_op.


[('When photography came along in the nineteenth century, painting was put in crisis',
  array([-5.2200427 , -0.00542181], dtype=float32),
  'IsNext'),
 ('The photograph, it seemed, did the work of imitating nature better than the painter ever could',
  array([-5.4660878e+00, -4.2366516e-03], dtype=float32),
  'IsNext')]